In [437]:
import random
import statistics
random.seed()

# Per Vanguard, actual rates of return  https://investor.vanguard.com/investor-resources-education/education/model-portfolio-allocation
# % bonds  max   min   average
# 100      .32  -.18  .050
#  90      .312 -.137 .058
#  80      .298 -.144 .064
#  70      .284 -.15  .071
#  60      .279 -.184 .077
#  50      .323 -.224 .082
#  40      .367 -.266 .088
#  30      .411 -.307 .092
#  20      .454 -.349 .097
#  10      .498 -.39  .101
#   0      .542 -.431 .105
#
#  My thinking is that safeNegg is in VTIP,CDs and SPAXX
#  negg is 80/20, which works out to around 65/35, but I'm using rates that work out closer to 50/50
#
#  Would be interesting to make rate vary dynamically within the historic range of actual AA

runs = 0
averageRate = .00
highRail = 100000
lowRail = 60000
rates = fictional_rates = [.01, .02, .03, .04, .05, .06, .05, .03, .024, .077, .075, .07, .07, .07,
         .082, .082, .08, .08,
         -.01, -.02, -.03, -.1, -.2, -.225, -.023, -.04, -.03, -.1, -.2, -.02, -.02, -.01,
         .09, .09, .09, .09, .10, .10, .10, .13, .15, .16, .17, .14, .15, .15, .17, .18, .19, .1, .1, .2, 
         .2, .25, .3, .3, .2, .2, .1, .1, .323]
pastRates = []
#This is a table of values produced by Claude to simulate the performance of AOA from years 1970 onward
# actual values are used from 2008 onward.
# I later had Gemini produce the same table, then adjusted by eye to an average of the two.  They weren't far off.
aoa_returns_pct = [
    -.0105, .18, .20, -.1252, -.2061, .3234, .1629, .0105, .1274, .1219,  # 1970-1979
    .2393, -.015, .1879, .2087, .0869, .3752, .3019, .0919, .1861, .2357,      # 1980-1989
    -.078, .2288, .0269, .1646, .0102, .2839, .1435, .2035, .2359, .1820,      # 1990-1999
    -.0507, -.1089, -.1339, .2737, .1315, .0691, .1669, .0796, -.28, .2579,   # 2000-2009
    .1674, -.0174, .1503, .2242, .0601, -.0108, .0777, .2004, -.0787, .2260,      # 2010-2019
    .1281, .1541, -.1623, .1827, .1355, .1962                                         # 2020-2025
]


wRate = .06
debug = True
rate = 0.08

def initialize(savings = 1600000, lowSpend = 0, highSpend = 0):
    global runs
    global rates
    global pastRates
    global negg
    global safeNegg
    global highRail
    global lowRail
    global averageRate
    global wRate
    
    pastRates = []
    runs = 0
    averageRate = .00
    #highRail = 100000
    #lowRail = 60000
    #highRail = 73000
    #lowRail = 72000
    total = savings
    if lowSpend > 0:
        lowRail = lowSpend
    else:
        lowRail = total*.045

    if highSpend > 0:
        highRail = highSpend
    else :
        highRail = lowRail*1.4
        
    #negg = total*.83
    #safeNegg = total*.17
    safeNegg = lowRail*5
    negg = total-safeNegg
    rates = aoa_returns_pct
    rate = random.choice(rates)
    wRate = .06
    

def status():
    print(f"Rate :{rate:.4f}, Average rate :{averageRate:.4f}")
    print(f"Long term :{negg:.2f}, Short term :{safeNegg:.2f}, Spend : {safeNegg*.3333:.2f}")
    print(f"Low rail :{lowRail:.2f}, High rail :{highRail:.2f}, Stock percentage :{(negg*.8)/(negg+safeNegg):.4f}")

   
    
    
def yearly():
    global negg
    global rate
    global wRate
    global safeNegg
    global highRail
    global lowRail
    global runs
    global averageRate
    global debug

   
    negg += negg*rate
    safeNegg += safeNegg*.02   #Assuming flat rate of growth for fixed income

    if(rate<wRate):
        if(rate>=0):
            wr = (rate*2+wRate)/3
        else :
            wr = .01  #Pretty arbitrary number.  Need logic to let this be 0 in some cases
    else :
        wr = wRate
        #  This doesn't build the bucket back fast enough for long term stagnation.
        #  A year 2000 retirement is just tough.
        #if(len(pastRates) > 1 and pastRates[-1] < 0) :   #this year was good, but last year was bad
        #    wr = wr*.5  #Let investments grow a bit in first year after down year.
        
    if debug: print(f"Rate :{rate:.4f}, Average Rate :{averageRate:.4f}")
    
    withdrawal = negg*wr
    if debug: print(f"Withdrawal rate :{wr}, amount :{withdrawal:.2f}")

    #if(withdrawal+safeNegg > highRail*4):
    #    withdrawal = (highRail*4)-safeNegg
    #elif(withdrawal+safeNegg < lowRail*2 and negg > (2*lowRail)):
    #    withdrawal = (lowRail*2)-safeNegg

    if(withdrawal+safeNegg > (lowRail*2+highRail*3)):
        withdrawal = (lowRail*2+highRail*3)-safeNegg
    elif(wr == wRate and withdrawal+safeNegg < lowRail*4 and (len(pastRates) > 1 and ((pastRates[-1]+rate)/2 > .1))):
        withdrawal = negg*(wr+.03)
    elif(withdrawal+safeNegg < lowRail*2 and negg > (2*lowRail)):
        withdrawal = (lowRail*2)-safeNegg
    


    if debug: print(f"Actual withdrawal :{withdrawal:.2f}")
    
        
    negg -= withdrawal

    safeNegg += withdrawal
    #spend = safeNegg*.25
    spend = safeNegg*.2
    if(spend > highRail): spend = highRail
    elif(spend < lowRail and safeNegg > lowRail):
        spend = lowRail
        
    safeNegg -= spend
    pastRates.append(rate)
    averageRate = statistics.mean(pastRates)
    runs+=1

    
    if debug: print(f"Long term :{negg:.2f}, Short term :{safeNegg:.2f}, Stock Percentage :{(negg*.8)/(safeNegg+negg):.4f}")
   
   # print(w, lowRail, highRail)
    if debug: print(f"Spend :{spend:.2f}, as percentage :{spend/(negg+safeNegg+spend):.4f}")
    
    
    rate = random.choice(rates)
    highRail = highRail*1.02  # This could match inflation if you want to keep spending more, or keep flat to assume SS kicking in.
    lowRail = lowRail*1.02
    #print(rate)
    if debug: print(f"Year :{runs}")

initialize()
yearly()
    


Rate :0.0800, Average Rate :0.0000
Withdrawal rate :0.06, amount :80352.00
Actual withdrawal :79200.00
Long term :1260000.00, Short term :357120.00, Stock Percentage :0.6233
Spend :89280.00, as percentage :0.0523
Year :1


In [404]:
debug = True
initialize(1600000)
status()

Rate :0.1315, Average rate :0.0000
Long term :1240000.00, Short term :360000.00, Spend : 119988.00
Low rail :72000.00, High rail :115200.00, Stock percentage :0.6200


In [417]:
#wRate = .03
#highRail = 40000
#lowRail = 10000
#rate = .17
yearly()
print(f" Guardrails : {lowRail:.2f} - {highRail:.2f}")

Rate :0.2737, Average Rate :0.0747
Withdrawal rate :0.06, amount :114837.28
Actual withdrawal :114837.28
Long term :1799117.39, Short term :318989.94, Stock Percentage :0.6795
Spend :91313.41, as percentage :0.0413
Year :13
 Guardrails : 93139.68 - 149023.48


In [438]:
# This simulation lets the rate be randomized, but adjusts guardrails as SS kicks in
initialize()
debug = False
#rate = .1

fail = 0
print("*****************************")
for j in range(1000):
    print("------------------------------------------")
    #initialize(1600000,95000)
    initialize(1600000)
    #status()
   
    for i in range(30):
        if(debug): print("*******************************")
        #rate = .06
        #if(i == 5):
        #    highRail -= 10000
        #    lowRail -= 5000
            #print(f"New guardrails :{lowRail}-{highRail}")
        #if(i == 10):
        #    highRail -= 40000
        #    lowRail -= 20000
            #print(f"New guardrails :{lowRail}-{highRail}")
 #   rate = .1
        yearly()
    if(negg+safeNegg < 500000): 
        fail += 1
        status()

print("--------------------")
print(f"Failures : {fail}")

*****************************
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
------------------------------------------
Rate :0.1629, Average rate :0.0822
Long term :143073.91, Short term :14488.53, Spend : 4829.03
Low rail :130418.03, High rail :182585.25, Stock percentage :0.7264
---------------------------------

In [436]:

debug = True
#rate = .1

fail = 0
print("*****************************")
# year 0 is 1970
for j in range(30,31):  #Year 2000, worst case
#for j in range(32):
    print(f"------------------{j+1970}------------------------")
    initialize(1600000, 75000, 96000)
    status()
    #highRail = 73000
    #lowRail = 72000
    for i in range(30):
        if(debug): print(f"*************{1970+j+i}******************")
        if((j+i)<len(aoa_returns_pct)):  # Set rate from actual returns, if available
            rate = aoa_returns_pct[j+i]
        else:
            rate = averageRate
            #print(f"Setting rate to :{rate}")
        # Allow for reducing withdrawals based on SS
        if(i == 5):
            highRail -= 10000
            lowRail -= 5000
            if debug: print(f"New guardrails :{lowRail}-{highRail}")
       
        if(i == 10):
            highRail -= 40000
            lowRail -= 20000
            if debug: print(f"New guardrails :{lowRail}-{highRail}")
 #   rate = .1
        yearly()
    if((negg+safeNegg) < 500000): fail += 1
    status()

print("--------------------")
print(f"Failures : {fail}")

*****************************
------------------2000------------------------
Rate :0.3019, Average rate :0.0000
Long term :1225000.00, Short term :375000.00, Spend : 124987.50
Low rail :75000.00, High rail :96000.00, Stock percentage :0.6125
*************2000******************
Rate :-0.0507, Average Rate :0.0000
Withdrawal rate :0.01, amount :11628.93
Actual withdrawal :11628.93
Long term :1151263.57, Short term :315303.14, Stock Percentage :0.6280
Spend :78825.79, as percentage :0.0510
Year :1
*************2001******************
Rate :-0.1089, Average Rate :-0.0507
Withdrawal rate :0.01, amount :10258.91
Actual withdrawal :10258.91
Long term :1015632.06, Short term :255368.11, Stock Percentage :0.6393
Spend :76500.00, as percentage :0.0568
Year :2
*************2002******************
Rate :-0.1339, Average Rate :-0.0798
Withdrawal rate :0.01, amount :8796.39
Actual withdrawal :8796.39
Long term :870842.54, Short term :191241.86, Stock Percentage :0.6559
Spend :78030.00, as percentage :

In [46]:
total = 1500000
negg = total*

932.2